In [3]:
# import pandas as pd
# import numpy as np 
# import matplotlib.pyplot as plt
# import os
# from PIL import Image
# import imagehash
# import torch
# from collections import Counter

In [1]:
# import matplotlib.pyplot as plt
# from PIL import Image
# import os

# # Get one image from each label
# label_0_image = train_df[train_df['label'] == 0].iloc[0]['file_name']
# label_1_image = train_df[train_df['label'] == 1].iloc[0]['file_name']

# # Paths
# label_0_path = os.path.join(train_images_path, label_0_image.replace("train_data/", ""))
# label_1_path = os.path.join(train_images_path, label_1_image.replace("train_data/", ""))

# # Display images
# fig, ax = plt.subplots(1, 2, figsize=(10, 5))
# for idx, (label, path) in enumerate([(0, label_0_path), (1, label_1_path)]):
#     try:
#         img = Image.open(path)
#         ax[idx].imshow(img)
#         ax[idx].set_title(f"Label {label}")
#         ax[idx].axis("off")
#     except FileNotFoundError:
#         print(f"File not found: {path}")
# plt.tight_layout()
# plt.show()


In [7]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torch.optim import Adam
from torch.amp import GradScaler, autocast
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torchvision.models import ResNet18_Weights
from tqdm import tqdm

# --- 1. Dataset Class ---
class ImageDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Reference file paths correctly
        if 'file_name' in self.dataframe.columns:
            img_name = os.path.join(self.root_dir, self.dataframe.iloc[idx]['file_name'].split('/')[-1])
        elif 'id' in self.dataframe.columns:
            img_name = os.path.join(self.root_dir, self.dataframe.iloc[idx]['id'].split('/')[-1])
        else:
            raise ValueError("Dataset does not contain valid file path column (file_name or id).")

        label = self.dataframe.iloc[idx]['label'] if 'label' in self.dataframe.columns else None
        try:
            image = Image.open(img_name).convert("RGB")
        except Exception as e:
            print(f"Error loading image {img_name}: {e}")
            return None, None

        if self.transform:
            image = self.transform(image)

        return image, int(label) if label is not None else None

# --- 2. Data Transformations ---
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# --- 3. Load Data ---
train_csv_path = "train.csv"  # Replace with actual path
#test_csv_path = "test.csv"  # Replace with actual path
train_dir = "train_data"  # Replace with your train data directory
#test_dir = "test_data"  # Replace with your test data directory

# Load the datasets
train_df = pd.read_csv(train_csv_path).sample(n=30000, random_state=42)
#test_df = pd.read_csv(test_csv_path).sample(n=100, random_state=42)

# # Validate training file paths
# missing_files_train = []
# for idx, row in train_df.iterrows():
#     img_path = os.path.join(train_dir, row["file_name"].split('/')[-1])
#     if not os.path.exists(img_path):
#         missing_files_train.append(img_path)

# if missing_files_train:
#     print(f"Missing training files: {missing_files_train}")
# else:
#     print("All training files exist.")

# # Validate testing file paths
# missing_files_test = []
# for idx, row in test_df.iterrows():
#     img_path = os.path.join(test_dir, row["id"].split('/')[-1])
#     if not os.path.exists(img_path):
#         missing_files_test.append(img_path)

# if missing_files_test:
#     print(f"Missing test files: {missing_files_test}")
# else:
#     print("All test files exist.")

train_dataset = ImageDataset(train_df, train_dir, transform=transform)
#test_dataset = ImageDataset(test_df, test_dir, transform=transform)

# DataLoader with reduced num_workers
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0, pin_memory=False)
#test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=0, pin_memory=False)

# --- 4. Define the Model ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 1)  # Single output, no sigmoid
model = model.to(device)

# --- 5. Loss, Optimizer, and Mixed Precision ---
criterion = nn.BCEWithLogitsLoss()
optimizer = Adam(model.parameters(), lr=1e-4)
scaler = GradScaler()

# --- 6. Training Loop ---
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", unit="batch")

    for batch_idx, (images, labels) in enumerate(progress_bar):
        if images is None or labels is None:  # Skip corrupted data
            continue

        images, labels = images.to(device), labels.to(device).float()

        optimizer.zero_grad()
        with autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):  # Mixed precision forward pass
            outputs = model(images).squeeze()
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}")

# model.eval()
# all_preds, all_labels = [], []

# with torch.no_grad():
#     for images, labels in tqdm(test_loader, desc="Testing", unit="batch"):
#         if images is None or labels is None:
#             continue

#         images, labels = images.to(device), labels.to(device).float()
#         outputs = model(images).squeeze()
#         preds = (outputs > 0).float()

#         all_preds.extend(preds.cpu().numpy())
#         all_labels.extend(labels.cpu().numpy())

# accuracy = accuracy_score(all_labels, all_preds)
# precision = precision_score(all_labels, all_preds)
# recall = recall_score(all_labels, all_preds)
# f1 = f1_score(all_labels, all_preds)

# print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

Epoch 1/5:  10%|▉         | 181/1875 [00:21<03:23,  8.33batch/s, loss=0.0893]


KeyboardInterrupt: 